# ADHD Dectection
[INTRODUCTION OF THE PROBLEM HERE]

## Dataset:
[ADD AND COMMENT ABOUT DATASETS HERE]

## Task:
[ADD TASK HERE]

## Integrants:
Name, Matricola, Email

- Jan Elfes,,

- Santiago Víquez Segura, 2048722, santiago.viquezsegura@studenti.unipd.it


## Dependencies

In [76]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import nibabel as nib
import pandas as pd
from config_local import helpers

from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

hp = helpers()

In [3]:
_data_path = hp.get_data_path()
print(_data_path)

data/


## Baseline

Before working with the FMRI data we will fit a logistic regression classifier on the phenotypic tabular data to come up with a baseline. We expect that any experiment using the FMRI data outperforms the following result.

In [39]:
phenotypic_df = pd.read_csv(_data_path + "NYU_phenotypic.csv")
phenotypic_df.head()

,ScanDir ID,Site,Gender,Age,Handedness,DX,Secondary Dx,ADHD Measure,ADHD Index,Inattentive,...,Performance IQ,Full2 IQ,Full4 IQ,Med Status,QC_Rest_1,QC_Rest_2,QC_Rest_3,QC_Rest_4,QC_Anatomical_1,QC_Anatomical_2
0,1000804,5,1.0,7.29,0.83,0,NaN,2,40,41,...,103,NaN,109,1,1.0,0.0,NaN,NaN,1.0,NaN
1,1023964,5,1.0,8.29,0.57,3,NaN,2,60,56,...,125,NaN,123,-999,1.0,0.0,NaN,NaN,1.0,1.0
2,1057962,5,1.0,8.78,-999.00,1,NaN,2,77,81,...,121,NaN,129,1,1.0,NaN,NaN,NaN,1.0,0.0
3,1099481,5,0.0,8.04,0.50,1,NaN,2,86,82,...,120,NaN,116,1,1.0,0.0,NaN,NaN,1.0,1.0
4,1127915,5,0.0,12.44,0.21,0,NaN,2,42,43,...,126,NaN,124,1,1.0,1.0,NaN,NaN,1.0,1.0


We are going to use `"Gender", "ADHD Index", "Age", "Handedness", "Inattentive", "Hyper/Impulsive", "Verbal IQ","Performance IQ", "Full4 IQ", "Med Status", "QC_Rest_1", "QC_Rest_2", "QC_Anatomical_1", "QC_Anatomical_2"` as predictors and use the token `-999` to fill and recognize NaN values.

In [43]:
features = ["Gender", "ADHD Index", "Age", "Handedness", 
            "Inattentive", "Hyper/Impulsive", "Verbal IQ",
           "Performance IQ", "Full4 IQ", "Med Status",
            "QC_Rest_1", "QC_Rest_2", "QC_Anatomical_1", 
            "QC_Anatomical_2"]

phenotypic_df = phenotypic_df[features + ["DX"]].fillna(-999)

In [99]:
X = phenotypic_df[features]
y = phenotypic_df.DX

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]
    
    clf = LogisticRegression(max_iter=5000,
                             solver="saga",
                             penalty="none",
                             random_state=42).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    score = f1_score(y_test, y_pred, average="weighted")
    scores.append(score)
    print(f"Fold {i} -- f1-Score (val): {score}")
print(f"\nAverage f1-Score (val): {np.mean(scores)}")

/Users/santiago/opt/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Fold 0 -- f1-Score (val): 0.6333177403546288
Fold 1 -- f1-Score (val): 0.7384038800705467
Fold 2 -- f1-Score (val): 0.6791931472782536
Fold 3 -- f1-Score (val): 0.7174205105239587
Fold 4 -- f1-Score (val): 0.663092463092463

Average f1-Score (val): 0.6862855482639703
